In [16]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
import scml
from scml import pandasx as pdx
from mylib.ner import NerModel
tim = scml.Timer()
tim.start()
os.environ["TOKENIZERS_PARALLELISM"] = "false"
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()
info = np.iinfo(np.int16)
print(f"int16, min={info.min}, max={info.max}")

int16, min=-32768, max=32767


In [17]:
dst_path = Path("models/ner/deberta_v3_large/20240309_092749")
ckpt_path = dst_path / "lightning_logs/version_8/checkpoints/epoch=2-step=22044-val_loss=0.00168.ckpt"

In [18]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)

In [19]:
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    for i in range(torch.cuda.device_count()):
        print(f"device={i}, {torch.cuda.get_device_name(i)}")
        print('Mem Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
        print('Mem Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')
else:
    print("cpu")

device=0, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB
device=1, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 4.9 GB
Mem Cached:    4.9 GB


In [20]:
%%time
#model = NerModel.load_from_checkpoint(ckpt_path)
model = NerModel.load_from_checkpoint(
    ckpt_path,
    pretrained_dir=Path("/mnt/c/huggingface/microsoft/deberta-v3-large"),
    lr=0,
    scheduler_conf=[],
)

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at /mnt/c/huggingface/microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CPU times: user 27.7 s, sys: 3.15 s, total: 30.8 s
Wall time: 38.8 s


In [21]:
%%time
model.model_to_save.save_pretrained(str(dst_path))

CPU times: user 2.29 s, sys: 242 ms, total: 2.53 s
Wall time: 7.76 s


In [22]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 0:00:46.602131
